In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud, STOPWORDS
import pyprind 
import warnings
warnings.filterwarnings('ignore')
import gc
import re
import gensim
from gensim.models import KeyedVectors
import operator
import string
import tensorflow as tf
import json
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
from zipfile import ZipFile
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Flatten,Dense, Embedding, Concatenate, Input, Dropout,Bidirectional, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D, LeakyReLU, Activation, LSTM, SpatialDropout1D, BatchNormalization
#Dense, Embedding, Bidirectional, CuDNNGRU, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, Input, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, BaseLogger
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
#from sklearn.metrics import precision_recall_fscore_support

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


/kaggle/input/paragram-300-sl999/paragram_300_sl999/README.txt
/kaggle/input/paragram-300-sl999/paragram_300_sl999/paragram_300_sl999.txt
/kaggle/input/paragram-300-sl999/paragram_300_sl999/paragram_300_sl999/README.txt
/kaggle/input/paragram-300-sl999/paragram_300_sl999/paragram_300_sl999/paragram_300_sl999.txt
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/google-quest-challenge/sample_submission.csv


In [2]:
train = pd.read_csv('../input/google-quest-challenge/train.csv', header=0,encoding='utf-8')
test = pd.read_csv('../input/google-quest-challenge/test.csv', header=0,encoding='utf-8')
df = pd.concat([train,test],axis=0,ignore_index=True)
print(f'''Train Shape: {train.shape}
Test Shape: {test.shape}
Df Shape:{df.shape}''')

Train Shape: (6079, 41)
Test Shape: (476, 11)
Df Shape:(6555, 41)


In [3]:
# df['treated_question_title'] = df['question_title'].progress_apply(lambda x: x.lower().replace('\n',' ').strip())
# df['treated_question_body'] = df['question_body'].progress_apply(lambda x: x.lower().replace('\n',' ').strip())
# df['treated_answer'] = df['answer'].progress_apply(lambda x: x.lower().replace('\n',' ').strip())

In [4]:
def separate_puncts(sentence, category):
    if category=='STACKOVERFLOW' or  category=='TECHNOLOGY':
        sentence = re.sub(r"(<.*?>)|(/*.*?\*/)|(for.*?\+\+)|(lt;.*?}})|(gt\s.*?}})|({\s.*?})|([&\s]lt;.*?gt[;\s])",' programming code ',sentence)
        sentence = re.sub(r"(programming code[\s\W]*)", ' programming code ',sentence)
    sentence = re.sub(r'http\S+', 'url', sentence)
    emojis=re.findall(r'\s(?::|;|=|\^)(?:-|\s)?(?:\)|\\|\/|\(|D|P|\^|\|)\s',sentence)
    for i in emojis:
        sentence = sentence.replace(i,'')
    sentence = re.sub(r"(\$\$.*?\$\$)", ' mathematical formula ', sentence)
    sentence = re.sub(r"((?<=\w)[^\s\w'?\-](?![^\s\w]))|([^\s\w'?\-](?![^\s\w])(?=\w))",' ',sentence)
    sentence = re.sub(r"((?<=\w)[.?\/\-])|([.?\/\-](?=\w))",' \g<1>\g<2> ',sentence)
    return ' '.join([sentence]+emojis)

In [5]:
def build_vocab(sentences, vocab={}, verbose=True):
    sentences = sentences.apply(lambda x: x.split()).values
    #vocab={}
    for sentence in pyprind.prog_bar(sentences):
        for word in sentence:
            try:
                vocab[word] +=1
            except KeyError:
                vocab[word] = 1
    return vocab

In [6]:
# def load_embed(file):
#     def get_coefs(word, *arr):
#         return word, np.array(arr, dtype='float16')
    
#     if file == '../input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
#         embeddings_index = dict(get_coefs(*o.split(' ')) for o in open(file,encoding='utf-8') if len(o)>100)
    
#     elif file == '../input/quora-insincere-questions-classification/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin':
#         embeddings_index = KeyedVectors.load_word2vec_format(file, binary=True)
    
#     elif file == '../input/quora-insincere-questions-classification/embeddings.zip/paragram_300_sl999/paragram_300_sl999.txt':
#         embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin') if len(o)>100)
        
#     elif file == '../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt':
#         embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
#     return embeddings_index



# glove = '../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt'
# paragram =  '..//input/quora-insincere-questions-classification/embeddings.zip/paragram_300_sl999/paragram_300_sl999.txt'
# wiki_news = '../input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
# google_embed = '../input/quora-insincere-questions-classification/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'

In [7]:
def load_embed(file, name):
    def get_coefs(word, *arr):
        return word, np.array(arr, dtype='float16')
    
    if name == 'wiki':
        embeddings_index = dict(get_coefs(*o.split(' ')) for o in open(file,encoding='utf-8') if len(o)>100)
    
    elif name == 'word2vec':
        embeddings_index = KeyedVectors.load_word2vec_format(file, binary=True)
    
    elif name == 'paragram':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin') if len(o)>100)
        
    elif name == 'glove':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

In [8]:
# with ZipFile('../input/quora-insincere-questions-classification/embeddings.zip','r') as f:
#     f.printdir()
#     f.extract('paragram_300_sl999/paragram_300_sl999.txt')
#     print('Done')

In [9]:
# print("Extracting GloVe embedding")
# embed_glove = load_embed(glove)
print("Extracting Paragram embedding")
embed_paragram = load_embed('../input/paragram-300-sl999/paragram_300_sl999/paragram_300_sl999/paragram_300_sl999.txt','paragram')

# print("Extracting Wiki embedding")
# embed_wiki = load_embed(wiki_news)
# print("Extracting Google embedding")
# embed_google = load_embed(google_embed)
print('Done')

Extracting Paragram embedding
Done


In [10]:
len(embed_paragram)

1703756

In [11]:
# os.remove('/kaggle/working/paragram_300_sl999/paragram_300_sl999.txt')

In [12]:
def check_coverage(vocab,embeddings_index):
    known_words={}
    unknown_words={}
    nb_known_words=0
    nb_unknown_words=0
    for word in vocab.keys():
        if word in embeddings_index:
            known_words[word]=embeddings_index[word]
            nb_known_words += vocab[word]
        elif word.capitalize() in embeddings_index:
            known_words[word] = embeddings_index[word.capitalize()]
            nb_known_words += vocab[word]
        elif word.lower() in embeddings_index:
            known_words[word] = embeddings_index[word.lower()]
            nb_known_words += vocab[word]
        elif word.upper() in embeddings_index:
            known_words[word] = embeddings_index[word.upper()]
            nb_known_words += vocab[word]
        else:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
    print(f'Found embeddings for {round((len(known_words)/len(vocab))*100,5)}% of the vocab\nFound embeddings for {round((nb_known_words/(nb_known_words+nb_unknown_words))*100,5)}% of all text')
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1),reverse=True)#[::-1]
    return unknown_words

In [13]:
# len(vocab),sorted(vocab.items(),key = operator.itemgetter(1),reverse=True)

In [14]:
contraction_mapping = {"\'":"'", #,"..":'',
    "Trump's" : 'trump is',"'cause": 'because',"â€™": "'",",cause": 'because',";cause": 'because',"ain't": 'am not',"ain,t": 'am not',
    "ain;t": 'am not',"ain't": 'am not',"ain’t": 'am not',"aren't": 'are not',"â€“": '-',"â€œ":'"',
    "aren,t": 'are not',"aren;t": 'are not',"aren't": 'are not',"aren’t": 'are not',"can't": 'cannot',"can't've": 'cannot have',"can,t": 'cannot',"can,t,ve": 'cannot have',
    "can;t": 'cannot',"can;t;ve": 'cannot have',
    "can't": 'cannot',"can't´ve": 'cannot have',"can’t": 'cannot',"can’t’ve": 'cannot have',
    "could've": 'could have',"could,ve": 'could have',"could;ve": 'could have',"couldn't": 'could not',"couldn't've": 'could not have',"couldn,t": 'could not',"couldn,t,ve": 'could not have',"couldn;t": 'could not',
    "couldn;t;ve": 'could not have',"couldn't": 'could not',
    "couldn't´ve": 'could not have',"couldn’t": 'could not',"couldn’t’ve": 'could not have',"could´ve": 'could have',
    "could’ve": 'could have',"didn't": 'did not',"didn,t": 'did not',"didn;t": 'did not',"didn't": 'did not',
    "didn’t": 'did not',"doesn't": 'does not',"doesn,t": 'does not',"doesn;t": 'does not',"doesn't": 'does not',
    "doesn’t": 'does not',"don't": 'do not',"don,t": 'do not',"don;t": 'do not',"don't": 'do not',"don’t": 'do not',
    "hadn't": 'had not',"hadn't've": 'had not have',"hadn,t": 'had not',"hadn,t,ve": 'had not have',"hadn;t": 'had not',
    "hadn;t;ve": 'had not have',"hadn't": 'had not',"hadn't´ve": 'had not have',"hadn’t": 'had not',"hadn’t’ve": 'had not have',"hasn't": 'has not',"hasn,t": 'has not',"hasn;t": 'has not',"hasn't": 'has not',"hasn’t": 'has not',
    "haven't": 'have not',"haven,t": 'have not',"haven;t": 'have not',"haven't": 'have not',"haven’t": 'have not',"he'd": 'he would',
    "he'd've": 'he would have',"he'll": 'he will',
    "he's": 'he is',"he,d": 'he would',"he,d,ve": 'he would have',"he,ll": 'he will',"he,s": 'he is',"he;d": 'he would',
    "he;d;ve": 'he would have',"he;ll": 'he will',"he;s": 'he is',"he'd": 'he would',"he'd've": 'he would have',"he´ll": 'he will',
    "he´s": 'he is',"he’d": 'he would',"he’d’ve": 'he would have',"he’ll": 'he will',"he’s": 'he is',"how'd": 'how did',"how'll": 'how will',
    "how's": 'how is',"how,d": 'how did',"how,ll": 'how will',"how,s": 'how is',"how;d": 'how did',"how;ll": 'how will',
    "how;s": 'how is',"how´d": 'how did',"how´ll": 'how will',"how´s": 'how is',"how’d": 'how did',"how’ll": 'how will',
    "how’s": 'how is',"i'd": 'i would',"i'll": 'i will',"i'm": 'i am',"i've": 'i have',"i,d": 'i would',"i,ll": 'i will',
    "i,m": 'i am',"i,ve": 'i have',"i;d": 'i would',"i;ll": 'i will',"i;m": 'i am',"i;ve": 'i have',"isn't": 'is not',
    "isn,t": 'is not',"isn;t": 'is not',"isn't": 'is not',"isn’t": 'is not',"it'd": 'it would',"it'll": 'it will',"It's":'it is',
    "it's": 'it is',"it,d": 'it would',"it,ll": 'it will',"it,s": 'it is',"it;d": 'it would',"it;ll": 'it will',"it;s": 'it is',"it´d": 'it would',"it´ll": 'it will',"it´s": 'it is',
    "it’d": 'it would',"it’ll": 'it will',"it’s": 'it is',
    "i´d": 'i would',"i´ll": 'i will',"i´m": 'i am',"i´ve": 'i have',"i’d": 'i would',"i’ll": 'i will',"i’m": 'i am',
    "i’ve": 'i have',"let's": 'let us',"let,s": 'let us',"let;s": 'let us',"let´s": 'let us',
    "let’s": 'let us',"ma'am": 'madam',"ma,am": 'madam',"ma;am": 'madam',"mayn't": 'may not',"mayn,t": 'may not',"mayn;t": 'may not',
    "mayn't": 'may not',"mayn’t": 'may not',"ma´am": 'madam',"ma’am": 'madam',"might've": 'might have',"might,ve": 'might have',"might;ve": 'might have',"mightn't": 'might not',"mightn,t": 'might not',"mightn;t": 'might not',"mightn't": 'might not',
    "mightn’t": 'might not',"might´ve": 'might have',"might’ve": 'might have',"must've": 'must have',"must,ve": 'must have',"must;ve": 'must have',
    "mustn't": 'must not',"mustn,t": 'must not',"mustn;t": 'must not',"mustn't": 'must not',"mustn’t": 'must not',"must´ve": 'must have',
    "must’ve": 'must have',"needn't": 'need not',"needn,t": 'need not',"needn;t": 'need not',"needn't": 'need not',"needn’t": 'need not',"oughtn't": 'ought not',"oughtn,t": 'ought not',"oughtn;t": 'ought not',
    "oughtn't": 'ought not',"oughtn’t": 'ought not',"sha'n't": 'shall not',"sha,n,t": 'shall not',"sha;n;t": 'shall not',"shan't": 'shall not',
    "shan,t": 'shall not',"shan;t": 'shall not',"shan't": 'shall not',"shan’t": 'shall not',"sha´n't": 'shall not',"sha’n’t": 'shall not',
    "she'd": 'she would',"she'll": 'she will',"she's": 'she is',"she,d": 'she would',"she,ll": 'she will',
    "she,s": 'she is',"she;d": 'she would',"she;ll": 'she will',"she;s": 'she is',"she´d": 'she would',"she´ll": 'she will',
    "she´s": 'she is',"she’d": 'she would',"she’ll": 'she will',"she’s": 'she is',"should've": 'should have',"should,ve": 'should have',"should;ve": 'should have',
    "shouldn't": 'should not',"shouldn,t": 'should not',"shouldn;t": 'should not',"shouldn't": 'should not',"shouldn’t": 'should not',"should´ve": 'should have',
    "should’ve": 'should have',"that'd": 'that would',"that's": 'that is',"that,d": 'that would',"that,s": 'that is',"that;d": 'that would',
    "that;s": 'that is',"that´d": 'that would',"that´s": 'that is',"that’d": 'that would',"that’s": 'that is',"there'd": 'there had',
    "there's": 'there is',"there,d": 'there had',"there,s": 'there is',"there;d": 'there had',"there;s": 'there is',
    "there´d": 'there had',"there´s": 'there is',"there’d": 'there had',"there’s": 'there is',
    "they'd": 'they would',"they'll": 'they will',"they're": 'they are',"they've": 'they have',
    "they,d": 'they would',"they,ll": 'they will',"they,re": 'they are',"they,ve": 'they have',"they;d": 'they would',"they;ll": 'they will',"they;re": 'they are',
    "they;ve": 'they have',"they´d": 'they would',"they´ll": 'they will',"they´re": 'they are',"they´ve": 'they have',"they’d": 'they would',"they’ll": 'they will',
    "they’re": 'they are',"they’ve": 'they have',"wasn't": 'was not',"wasn,t": 'was not',"wasn;t": 'was not',"wasn't": 'was not',
    "wasn’t": 'was not',"we'd": 'we would',"we'll": 'we will',"we're": 'we are',"we've": 'we have',"we,d": 'we would',"we,ll": 'we will',
    "we,re": 'we are',"we,ve": 'we have',"we;d": 'we would',"we;ll": 'we will',"we;re": 'we are',"we;ve": 'we have',
    "weren't": 'were not',"weren,t": 'were not',"weren;t": 'were not',"weren't": 'were not',"weren’t": 'were not',"we´d": 'we would',"we´ll": 'we will',
    "we´re": 'we are',"we´ve": 'we have',"we’d": 'we would',"we’ll": 'we will',"we’re": 'we are',"we’ve": 'we have',"what'll": 'what will',"what're": 'what are',"what's": 'what is',
    "what've": 'what have',"what,ll": 'what will',"what,re": 'what are',"what,s": 'what is',"what,ve": 'what have',"what;ll": 'what will',"what;re": 'what are',
    "what;s": 'what is',"what;ve": 'what have',"what´ll": 'what will',
    "what´re": 'what are',"what´s": 'what is',"what´ve": 'what have',"what’ll": 'what will',"what’re": 'what are',"what’s": 'what is',
    "what’ve": 'what have',"where'd": 'where did',"where's": 'where is',"where,d": 'where did',"where,s": 'where is',"where;d": 'where did',
    "where;s": 'where is',"where´d": 'where did',"where´s": 'where is',"where’d": 'where did',"where’s": 'where is',
    "who'll": 'who will',"who's": 'who is',"who,ll": 'who will',"who,s": 'who is',"who;ll": 'who will',"who;s": 'who is',
    "who´ll": 'who will',"who´s": 'who is',"who’ll": 'who will',"who’s": 'who is',"won't": 'will not',"won,t": 'will not',"won;t": 'will not',
    "won't": 'will not',"won't": 'will not',"wouldn't": 'would not',"wouldn,t": 'would not',"wouldn;t": 'would not',"wouldn't": 'would not',
    "wouldn't": 'would not',"you'd": 'you would',"you'll": 'you will',"you're": 'you are',"you,d": 'you would',"you,ll": 'you will',
    "you,re": 'you are',"you;d": 'you would',"you;ll": 'you will',
    "you;re": 'you are',"you´d": 'you would',"you´ll": 'you will',"you´re": 'you are',"you’d": 'you would',"you’ll": 'you will',"you’re": 'you are',
    "´cause": 'because',"’cause": 'because',"you've": "you have","could'nt": 'could not',
    "havn't": 'have not',"here’s": "here is","i'm": 'i am' ,"i'l": "i will","i'v": 'i have',"wan't": 'want',"was'nt": "was not","who'd": "who would",
    "who're": "who are","who've": "who have","why'd": "why would","would've": "would have","y'all": "you all","y'know": "you know","you.i": "you i",
    "your'e": "you are","arn't": "are not","agains't": "against","c'mon": "common","doens't": "does not","don't": "do not","dosen't": "does not",
    "dosn't": "does not","shoudn't": "should not","that'll": "that will","there'll": "there will","there're": "there are",
    "this'll": "this all","u're": "you are", "ya'll": "you all","you'r": "you are","you’ve": "you have","d'int": "did not","did'nt": "did not","din't": "did not","dont't": "do not","gov't": "government",
    "i'ma": "i am","is'nt": "is not","‘I":'I',"ᴀɴᴅ":'and',"ᴛʜᴇ":'the',"ʜᴏᴍᴇ":'home',"ᴜᴘ":'up',"ʙʏ":'by',"ᴀᴛ":'at',"…and":'and',"civilbeat":'civil beat',"TrumpCare":'Trump care',"Trumpcare":'Trump care', "OBAMAcare":'Obama care',"ᴄʜᴇᴄᴋ":'check',"ғᴏʀ":'for',"ᴛʜɪs":'this',"ᴄᴏᴍᴘᴜᴛᴇʀ":'computer',"ᴍᴏɴᴛʜ":'month',"ᴡᴏʀᴋɪɴɢ":'working',"ᴊᴏʙ":'job',"ғʀᴏᴍ":'from',"Sᴛᴀʀᴛ":'start',"gubmit":'submit',"CO₂":'carbon dioxide',"ғɪʀsᴛ":'first',"ᴇɴᴅ":'end',"ᴄᴀɴ":'can',"ʜᴀᴠᴇ":'have',"ᴛᴏ":'to',"ʟɪɴᴋ":'link',"ᴏғ":'of',"ʜᴏᴜʀʟʏ":'hourly',"ᴡᴇᴇᴋ":'week',"ᴇɴᴅ":'end',"ᴇxᴛʀᴀ":'extra',"Gʀᴇᴀᴛ":'great',"sᴛᴜᴅᴇɴᴛs":'student',"sᴛᴀʏ":'stay',"ᴍᴏᴍs":'mother',"ᴏʀ":'or',"ᴀɴʏᴏɴᴇ":'anyone',"ɴᴇᴇᴅɪɴɢ":'needing',"ᴀɴ":'an',"ɪɴᴄᴏᴍᴇ":'income',
    "ʀᴇʟɪᴀʙʟᴇ":'reliable',"ғɪʀsᴛ":'first',"ʏᴏᴜʀ":'your',"sɪɢɴɪɴɢ":'signing',"ʙᴏᴛᴛᴏᴍ":'bottom',"ғᴏʟʟᴏᴡɪɴɢ":'following',"Mᴀᴋᴇ":'make',
    "ᴄᴏɴɴᴇᴄᴛɪᴏɴ":'connection',"ɪɴᴛᴇʀɴᴇᴛ":'internet',"financialpost":'financial post', "ʜaᴠᴇ":' have ', "ᴄaɴ":' can ', "Maᴋᴇ":' make ', "ʀᴇʟɪaʙʟᴇ":' reliable ', "ɴᴇᴇᴅ":' need ',
    "ᴏɴʟʏ":' only ', "ᴇxᴛʀa":' extra ', "aɴ":' an ', "aɴʏᴏɴᴇ":' anyone ', "sᴛaʏ":' stay ', "Sᴛaʀᴛ":' start', "SHOPO":'shop',
    " :-/ ":'Perplexed smilee'," :/ ":'Perplexed smilee'," ;-/ ":'Perplexed'," ;/ ":'Perplexed', " ;/ ":'Perplexed', " :/ ":'Perplexed',
    " =( ":'Sad'," :-( ":'Sad'," :( ":'Sad', "=gt":'=>', "=gt":'=>', "n't":" not", "'s": " is", 'amp;amp':'ampersand', '..':'', '////':'', 'nbsp;':'',
    '\\\\':'',"usepackage" : "use package",'instrumentsettingsid':'instrumental settings id','rippleshaderProgram' : 'ripple shader program','shaderprogramconstants':'shader program constants','storedElements':'stored elements','stackSize' : 'stack size'
                #'_':' '
                   
    }

In [15]:
#bar = pyprind.ProgBar(df.shape[0], bar_char='█')
def clean_contractions(text, mapping):
    #text = text.split()
    #text_val = text.lower()
    #map_val = 
    for word in mapping.keys():
        word_val = word.lower()
        if word in text:
            text = text.replace(word, mapping[word])
        elif word_val in text:
            text = text.replace(word_val, mapping[word])
#         elif word.lower() in text:
#             text = text.replace(word, mapping[word])
#         elif word.capitalize() in text:
#             text = text.replace(word, mapping[word])
#         elif word.upper() in text:
#             text = text.replace(word, mapping[word])

#     bar.update()
    return text

In [16]:
# df['treated_question_title'] = df['treated_question_title'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))
# df['treated_question_body'] = df['treated_question_body'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))
# df['treated_answer'] = df['treated_answer'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))

In [17]:
punct = "-?!.,#$%\*+<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': ' divided by ', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [18]:
len(embed_paragram['smilee'])

300

In [19]:
def unknown_punct(embed, punct):
    unknown_punct=''
    for val in punct:
        if val not in embed:
            unknown_punct += val+' '
    return unknown_punct
# print(f'Glove:\n{unknown_punct(embed_glove, punct)}')
# print(f'Paragram:\n{unknown_punct(embed_paragram, punct)}')
# print(f'Wiki:\n{unknown_punct(embed_wiki, punct)}')
# print(f'Google:\n{unknown_punct(embed_google, punct)}')

In [20]:
def clean_puncts(text, mapping, punct):
    for p in punct_mapping.keys():
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
#     for word in text:
#         for p in '!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~':
#             word = word.replace(p, f' {p} ')
    
#     specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
#     for p in specials.keys():
#         text = text.replace(p, specials[p])
#    bar.update()    
    return text

# df['treated_question_title'] = df['treated_question_title'].progress_apply(lambda x: clean_puncts(x, punct_mapping, punct))
# df['treated_question_body'] = df['treated_question_body'].progress_apply(lambda x: clean_puncts(x, punct_mapping, punct))
# df['treated_answer'] = df['treated_answer'].progress_apply(lambda x: clean_puncts(x, punct_mapping, punct))



In [21]:
df['treated_question_title'] = df['question_title'].apply(lambda x: x.lower().replace('\n',' ').strip())
df['treated_question_body'] = df['question_body'].apply(lambda x: x.lower().replace('\n',' ').strip())
df['treated_answer'] = df['answer'].apply(lambda x: x.lower().replace('\n',' ').strip())
print('Stripping Done')

df['treated_question_title'] = df[['treated_question_title','category']].progress_apply(lambda x: separate_puncts(x['treated_question_title'], x['category']),axis=1)
df['treated_question_body'] = df[['treated_question_body','category']].progress_apply(lambda x: separate_puncts(x['treated_question_body'], x['category']),axis=1)
df['treated_answer'] = df[['treated_answer','category']].progress_apply(lambda x: separate_puncts(x['treated_answer'], x['category']),axis=1)
print('Cleaning Data Done')

df['treated_question_title'] = df['treated_question_title'].apply(lambda x: clean_contractions(x, contraction_mapping))
df['treated_question_body'] = df['treated_question_body'].apply(lambda x: clean_contractions(x, contraction_mapping))
df['treated_answer'] = df['treated_answer'].apply(lambda x: clean_contractions(x, contraction_mapping))
print('Contraction Mapping Done')

df['treated_question_title'] = df['treated_question_title'].apply(lambda x: clean_puncts(x, punct_mapping, punct))
df['treated_question_body'] = df['treated_question_body'].apply(lambda x: clean_puncts(x, punct_mapping, punct))
df['treated_answer'] = df['treated_answer'].apply(lambda x: clean_puncts(x, punct_mapping, punct))
print('Clean Punctuation Done')

Stripping Done



Cleaning Data Done
Contraction Mapping Done
Clean Punctuation Done


In [22]:
vocab = build_vocab(df['treated_question_title'])
print(len(vocab))
vocab = build_vocab(df['treated_question_body'], vocab=vocab)
print(len(vocab))
vocab = build_vocab(df['treated_answer'], vocab=vocab)
print(len(vocab))
# print("Glove : ")
# oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)

# print("Wiki : ")
# oov_glove = check_coverage(vocab, embed_wiki)
# print("Google : ")
# oov_paragram = check_coverage(vocab, embed_google)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


7949


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


32198


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


57088
Paragram : 
Found embeddings for 66.12248% of the vocab
Found embeddings for 97.35993% of all text


In [23]:
import traceback
traceback.extract_stack()[-2]

<FrameSummary file /opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py, line 3326 in run_code>

In [24]:
find = re.compile(r"^[^.]*")
df['web_category'] = df['url'].apply(lambda x: re.findall(find, x)[0].replace('http://',''))
df[['url', 'web_category']]

,url,web_category
0,http://photo.stackexchange.com/questions/9169/...,photo
1,http://rpg.stackexchange.com/questions/47820/w...,rpg
2,http://electronics.stackexchange.com/questions...,electronics
3,http://judaism.stackexchange.com/questions/551...,judaism
4,http://graphicdesign.stackexchange.com/questio...,graphicdesign
...,...,...
6550,http://tex.stackexchange.com/questions/165120/...,tex
6551,http://stackoverflow.com/questions/16872513/al...,stackoverflow
6552,http://dba.stackexchange.com/questions/105287/...,dba
6553,http://boardgames.stackexchange.com/questions/...,boardgames


In [25]:
df.groupby(['category','web_category']).size()

category    web_category
CULTURE     anime            29
            bicycles         77
            boardgames       71
            christianity     62
            ell              89
                           ... 
TECHNOLOGY  unix            119
            ux              104
            webapps          48
            webmasters       56
            wordpress       101
Length: 63, dtype: int64

In [26]:
for i in df['category'].unique():
    print(i, len(df[df['category']==i]['web_category'].unique()))
    

LIFE_ARTS 9
CULTURE 12
SCIENCE 12
STACKOVERFLOW 1
TECHNOLOGY 29


In [27]:
dums = pd.get_dummies(df[['category','web_category']])
df = pd.concat([df,dums], axis=1)

In [28]:
# df.loc[500, ['category','web_category','category_STACKOVERFLOW','web_category_stackoverflow']]

df.drop(columns=['question_title','question_body','answer', 'category','web_category'],inplace=True)

In [29]:
X_train = df[df['answer_helpful'].notna()]
X_test = df[df['answer_helpful'].isna()]

In [30]:
title_max, title_mean = df['treated_question_title'].apply(lambda x: x.split()).map(len).max(), df['treated_question_title'].apply(lambda x: x.split()).map(len).mean()
body_max, body_mean = df['treated_question_body'].apply(lambda x: x.split()).map(len).max(), df['treated_question_body'].apply(lambda x: x.split()).map(len).mean() 
ans_max, ans_mean = df['treated_answer'].apply(lambda x: x.split()).map(len).max(), df['treated_answer'].apply(lambda x: x.split()).map(len).mean() 
print(f'Df Max Title Size is:\t{title_max}\twith average size being:\t{title_mean}')
print(f'Df Max Body Size is:\t{body_max}\twith average size being:\t{body_mean}')
print(f'Df Max Ans Size is:\t{ans_max}\twith average size being:\t{ans_mean}')

Df Max Title Size is:	34	with average size being:	10.03203661327231
Df Max Body Size is:	2823	with average size being:	139.6335621662853
Df Max Ans Size is:	4152	with average size being:	145.89641495041954


In [31]:
title_max, title_mean = X_train['treated_question_title'].apply(lambda x: x.split()).map(len).max(), X_train['treated_question_title'].apply(lambda x: x.split()).map(len).mean()
body_max, body_mean = X_train['treated_question_body'].apply(lambda x: x.split()).map(len).max(), X_train['treated_question_body'].apply(lambda x: x.split()).map(len).mean() 
ans_max, ans_mean = X_train['treated_answer'].apply(lambda x: x.split()).map(len).max(), X_train['treated_answer'].apply(lambda x: x.split()).map(len).mean() 
print(f'X_train Max Title Size is:\t{title_max}\twith average size being:\t{title_mean}')
print(f'X_train Max Body Size is:\t{body_max}\twith average size being:\t{body_mean}')
print(f'X_train Max Ans Size is:\t{ans_max}\twith average size being:\t{ans_mean}')

X_train Max Title Size is:	34	with average size being:	10.061523276854746
X_train Max Body Size is:	2823	with average size being:	139.8781049514723
X_train Max Ans Size is:	4152	with average size being:	145.88221746997863


In [32]:
title_max, title_mean = X_test['treated_question_title'].apply(lambda x: x.split()).map(len).max(), X_test['treated_question_title'].apply(lambda x: x.split()).map(len).mean()
body_max, body_mean = X_test['treated_question_body'].apply(lambda x: x.split()).map(len).max(), X_test['treated_question_body'].apply(lambda x: x.split()).map(len).mean() 
ans_max, ans_mean = X_test['treated_answer'].apply(lambda x: x.split()).map(len).max(), X_test['treated_answer'].apply(lambda x: x.split()).map(len).mean() 
print(f'X_test Max Title Size is:\t{title_max}\twith average size being:\t{title_mean}')
print(f'X_test Max Body Size is:\t{body_max}\twith average size being:\t{body_mean}')
print(f'X_test Max Answ Size is:\t{ans_max}\twith average size being:\t{ans_mean}')

X_test Max Title Size is:	33	with average size being:	9.655462184873949
X_test Max Body Size is:	1604	with average size being:	136.51050420168067
X_test Max Answ Size is:	1586	with average size being:	146.07773109243698


In [33]:
maxlen = 250
length_to_keep = 1000000

In [34]:
def length_maker(text, maxlen=maxlen):
    text = text.split()
    if len(text)>250:
        text = text[:125]+text[-125:]
    return ' '.join(text)

In [35]:
df['treated_question_title'] = df['treated_question_title'].progress_apply(lambda x: length_maker(x))
df['treated_question_body'] = df['treated_question_body'].progress_apply(lambda x: length_maker(x))
df['treated_answer'] = df['treated_answer'].progress_apply(lambda x: length_maker(x))

In [36]:
X_train['treated_question_title'] = X_train['treated_question_title'].progress_apply(lambda x: length_maker(x))
X_train['treated_question_body'] = X_train['treated_question_body'].progress_apply(lambda x: length_maker(x))
X_train['treated_answer'] = X_train['treated_answer'].progress_apply(lambda x: length_maker(x))

In [37]:
X_test['treated_question_title'] = X_test['treated_question_title'].apply(lambda x: length_maker(x))
X_test['treated_question_body'] = X_test['treated_question_body'].apply(lambda x: length_maker(x))
X_test['treated_answer'] = X_test['treated_answer'].apply(lambda x: length_maker(x))

In [38]:
title_max, title_mean = X_test['treated_question_title'].apply(lambda x: x.split()).map(len).max(), X_test['treated_question_title'].apply(lambda x: x.split()).map(len).mean()
body_max, body_mean = X_test['treated_question_body'].apply(lambda x: x.split()).map(len).max(), X_test['treated_question_body'].apply(lambda x: x.split()).map(len).mean() 
ans_max, ans_mean = X_test['treated_answer'].apply(lambda x: x.split()).map(len).max(), X_test['treated_answer'].apply(lambda x: x.split()).map(len).mean() 
print(f'X_test Max Title Size is:\t{title_max}\twith average size being:\t{title_mean}')
print(f'X_test Max Body Size is:\t{body_max}\twith average size being:\t{body_mean}')
print(f'X_test Max Answ Size is:\t{ans_max}\twith average size being:\t{ans_mean}')

X_test Max Title Size is:	33	with average size being:	9.655462184873949
X_test Max Body Size is:	250	with average size being:	116.86974789915966
X_test Max Answ Size is:	250	with average size being:	115.94957983193277


In [39]:
cnt=0
cat_cols=[]
for i in X_train.columns:
    
    if i.startswith('category') or i.startswith('web_category'):
        cnt+=1
        cat_cols.append(i)
print(cat_cols, len(cat_cols))

['category_CULTURE', 'category_LIFE_ARTS', 'category_SCIENCE', 'category_STACKOVERFLOW', 'category_TECHNOLOGY', 'web_category_academia', 'web_category_android', 'web_category_anime', 'web_category_apple', 'web_category_askubuntu', 'web_category_bicycles', 'web_category_biology', 'web_category_blender', 'web_category_boardgames', 'web_category_chemistry', 'web_category_christianity', 'web_category_codereview', 'web_category_cooking', 'web_category_crypto', 'web_category_cs', 'web_category_dba', 'web_category_diy', 'web_category_drupal', 'web_category_dsp', 'web_category_electronics', 'web_category_ell', 'web_category_english', 'web_category_expressionengine', 'web_category_gamedev', 'web_category_gaming', 'web_category_gis', 'web_category_graphicdesign', 'web_category_judaism', 'web_category_magento', 'web_category_math', 'web_category_mathematica', 'web_category_mathoverflow', 'web_category_mechanics', 'web_category_meta', 'web_category_money', 'web_category_movies', 'web_category_musi

In [40]:
targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'    
    ]

In [41]:
y_train = X_train[targets]
X_train.drop(columns=targets+['answer_user_name', 'answer_user_page', 'host', 'qa_id', 'question_user_name', 'question_user_page', 'url'], inplace=True)

In [42]:

y_test = X_test[targets]
X_test.drop(columns=targets+['answer_user_name', 'answer_user_page', 'host', 'qa_id', 'question_user_name', 'question_user_page', 'url'], inplace=True)

In [43]:
X_test.shape, X_train.shape

((476, 67), (6079, 67))

In [44]:
X_train.columns

Index(['treated_question_title', 'treated_question_body', 'treated_answer',
       'category_CULTURE', 'category_LIFE_ARTS', 'category_SCIENCE',
       'category_STACKOVERFLOW', 'category_TECHNOLOGY',
       'web_category_academia', 'web_category_android', 'web_category_anime',
       'web_category_apple', 'web_category_askubuntu', 'web_category_bicycles',
       'web_category_biology', 'web_category_blender',
       'web_category_boardgames', 'web_category_chemistry',
       'web_category_christianity', 'web_category_codereview',
       'web_category_cooking', 'web_category_crypto', 'web_category_cs',
       'web_category_dba', 'web_category_diy', 'web_category_drupal',
       'web_category_dsp', 'web_category_electronics', 'web_category_ell',
       'web_category_english', 'web_category_expressionengine',
       'web_category_gamedev', 'web_category_gaming', 'web_category_gis',
       'web_category_graphicdesign', 'web_category_judaism',
       'web_category_magento', 'web_category_m

In [45]:
all_text = pd.concat([X_train['treated_question_body'],X_train['treated_answer'],X_test['treated_question_body'],X_test['treated_answer'],X_train["treated_question_title"],X_test["treated_question_title"]])

In [46]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=train['category'])

In [47]:

t=Tokenizer(num_words=length_to_keep, filters=' ')
t.fit_on_texts(all_text)
def tokenize_treated_data(text, text_test, maxlen=maxlen, t=t):
#     t=Tokenizer(num_words=length_to_keep, filters=' ')
#     t.fit_on_texts(text)
    text = t.texts_to_sequences(text)
    text_test = t.texts_to_sequences(text_test)
    text = pad_sequences(text, maxlen=maxlen)
    text_test = pad_sequences(text_test, maxlen=maxlen)
    return text, t.word_index, text_test


In [48]:
train_question_body, word_index, test_question_body = tokenize_treated_data(X_tr['treated_question_body'],
                                                                                          X_test['treated_question_body']
                                                                                         )
_, _, val_question_body = tokenize_treated_data(X_tr['treated_question_body'],
                                                                                          X_val['treated_question_body']
                                                                                         )

In [49]:
train_answer, _, test_answer = tokenize_treated_data(X_tr['treated_answer'],
                                                                                          X_test['treated_answer']
                                                                                         )

_, _, val_answer = tokenize_treated_data(X_train['treated_answer'],
                                                                                          X_val['treated_answer']
                                                                                         )

In [50]:
train_question_title, _, test_question_title = tokenize_treated_data(X_tr['treated_question_title'],
                                                                                          X_test['treated_question_title']
                                                                                         )

_, _, val_question_title = tokenize_treated_data(X_tr['treated_question_title'],
                                                                                          X_val['treated_question_title']
                                                                                         )

In [51]:
# del answer_word_index
# gc.collect()

In [52]:
def embed_matrix(embed_paragram = embed_paragram, word_index=word_index, length_to_keep = length_to_keep):
    embeddings = np.stack(embed_paragram.values())
    
#    embeddings_mean, embeddings_std = embeddings.mean(), embeddings.std(ddof=1)
#    print(embeddings_mean)
     
    embeddings_shape = embeddings.shape[1]
    embedding_matrix = np.zeros((length_to_keep, 300))
    
    for word, i in word_index.items():
        if i >= length_to_keep:
            continue
        embeddings_vector = embed_paragram.get(word)
        if embeddings_vector is not None:
            embedding_matrix[i] = embeddings_vector
    return embedding_matrix

In [53]:
embedding_mat = embed_matrix(embed_paragram, word_index, length_to_keep)
# embedding_question_title = embed_matrix(embed_paragram, question_title_word_index, length_to_keep)
# embedding_answer = embed_matrix(embed_paragram, answer_word_index, length_to_keep)

In [54]:
# del question_body_word_index,question_title_word_index, answer_word_index, embed_paragram
# gc.collect()

In [55]:
from tensorflow.keras.layers import Add
def make_model(embedding_matrices=embedding_mat, embed_size=300, loss='binary_crossentropy'):
    input_title = Input(shape=(maxlen,), name='title')
    input_body = Input(shape=(maxlen,), name='body')
    input_answer = Input(shape=(maxlen,), name='answer')
    input_category = Input(shape=(len(cat_cols),),name='categorical')
    GRU_nums=128
    ##Answer
    
    allwords = Concatenate(axis=1)([input_title,input_body, input_category, input_answer])
    x = Embedding(length_to_keep, embed_size, weights=[embedding_matrices],trainable=False)(allwords)
    ans_lstm = Bidirectional(LSTM(GRU_nums, return_sequences=True),merge_mode='sum')(x)
    
    ##Body
#     y = Embedding(length_to_keep, embed_size, weights=[embedding_matrices['body']],trainable=False)(input_body)
    ans_lstm2 = Bidirectional(LSTM(GRU_nums, return_sequences=True),merge_mode='sum')(ans_lstm)
    
    ##Title
#     z = Embedding(length_to_keep, embed_size, weights=[embedding_matrices['title']],trainable=False)(input_title)
    ans_lstm3 = Bidirectional(LSTM(GRU_nums, return_sequences=True), merge_mode='sum')(ans_lstm2)
    
#     conc = Concatenate(axis=2)([title_lstm, body_lstm, ans_lstm])
#     lstm1 = Bidirectional(LSTM(GRU_nums, return_sequences=False))(conc)
#     conc2 = Concatenate(axis=1)([lstm1, input_category])
#     lstm2 = Bidirectional(LSTM(GRU_nums, return_sequences=False))(conc2)
    maxp = Concatenate(axis=1)([GlobalMaxPooling1D()(ans_lstm), GlobalMaxPooling1D()(ans_lstm2), GlobalMaxPooling1D()(ans_lstm3)])
    avgp = Concatenate(axis=1)([GlobalAveragePooling1D()(ans_lstm), GlobalAveragePooling1D()(ans_lstm2), GlobalAveragePooling1D()(ans_lstm3)])
    conc = Concatenate(axis=1)([maxp, avgp])
    dense1 = Add()([Dense(GRU_nums*6)(conc),conc])
    dense1  = LeakyReLU()(dense1)
    dense2 = Add()([Dense(GRU_nums*6, activation = 'relu')(dense1),conc])
    dense2  = LeakyReLU()(dense2)
    output = Dense(30, activation='sigmoid')(dense2)
    model = Model(inputs=[input_title, input_body, input_answer, input_category], outputs=output)
    model.compile(loss=loss, optimizer=Adam(lr=1e-3),metrics=['accuracy'])
    return model

In [56]:
# a= Input(shape=(250,300))
# b =Input(shape=(250,300))
# c = Input(shape=(250,300))
# #d = Input(shape=(250,300))
# GRU_nums=64
# #x= Concatenate(axis=2)([b, c])

# o_1 = Bidirectional(LSTM(GRU_nums, return_sequences=True),)(a)
# o_2 = Bidirectional(LSTM(GRU_nums, return_sequences=True))(b)
# o_3 = Bidirectional(LSTM(16, return_sequences=True))(c)
# conc=GlobalMaxPooling1D()(o_2)
# conc2=GlobalAveragePooling1D()(o_2)
# conc = Concatenate(axis=1)([conc, conc2])

# # o_4 = Bidirectional(LSTM(128, return_sequences=False))(conc)

# model = Model(inputs=[a,b,c], outputs=conc)
# #conc = Concatenate(axis=1)([o_1, o_2])
# model.summary()

In [57]:
model = make_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title (InputLayer)              [(None, 250)]        0                                            
__________________________________________________________________________________________________
body (InputLayer)               [(None, 250)]        0                                            
__________________________________________________________________________________________________
categorical (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
answer (InputLayer)             [(None, 250)]        0                                            
______________________________________________________________________________________________

In [58]:
class TrainingMonitor(BaseLogger):
    def __init__(self, figPath=None, jsonPath=None, startAt=0):
        super().__init__()
        self.figPath = figPath
        self.jsonPath = jsonPath
        self.startAt = startAt
        
    def on_train_begin(self,logs={}):
        self.H = {}
        
        if self.jsonPath is not None:
            if os.path.exists(self.jsonPath):
                self.H = json.loads(open(self.jsonPath).read())
                
                if self.startAt>0:
                    for k in self.H:
                        H[k] = self.H[k][:self.startAt]
    
    def on_epoch_end(self, epoch, logs={}):
        for k,v in logs.items():
            orig_list = self.H.get(k,[])
            orig_list.append(float(v))
            self.H[k] = orig_list
            
        if self.jsonPath is not None:
            with open(self.jsonPath, 'w') as f:
                f.write(json.dumps(self.H))
                
        
        if len(self.H)>1:
            num = np.arange(0, len(self.H['loss']))
            fig = plt.figure(figsize=(12,8))
            sns.lineplot(x=num, y=self.H['loss'], label = 'Train Loss')
            sns.lineplot(x=num, y=self.H['accuracy'], label = 'Train Accuracy')
            sns.lineplot(x=num, y=self.H['val_loss'], label = 'Val Loss')
            sns.lineplot(x=num, y=self.H['val_accuracy'], label = 'Val Accuracy')
            plt.title('Training Loss and Accuracy [Epoch {}]')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy/Loss')
            plt.legend(loc='best')
            plt.savefig(self.figPath)
#             plt.show()
            plt.close()

In [59]:
earlystopping = EarlyStopping(monitor='val_loss', patience=8, mode='min', verbose=1, min_delta=0.01, restore_best_weights=True)
modelcheckpoint = ModelCheckpoint(filepath='/kaggle/working/best_model.hdf5', monitor='val_loss', mode='min',
                                  verbose=1, min_delta=0.04, save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1,factor=0.1, mode='min', min_lr=1e-6)
trainingmonitor = TrainingMonitor(figPath='/kaggle/working/training.png', jsonPath='/kaggle/working/training.json')

In [60]:
H = model.fit([train_question_title, train_question_body, train_answer, X_tr[cat_cols]],
              y_tr, 
              validation_data=([val_question_title, val_question_body, val_answer, X_val[cat_cols]],y_val),
              epochs=30,
              batch_size=128,
              callbacks = [trainingmonitor, modelcheckpoint, reduceLR, earlystopping],
              verbose=1
             )

Train on 5471 samples, validate on 608 samples
Epoch 1/30
5376/5471 [============================>.] - ETA: 9s - loss: 0.4219 - accuracy: 0.5540 
Epoch 00001: val_loss improved from inf to 0.40345, saving model to /kaggle/working/best_model.hdf5
5471/5471 [==============================] - 571s 104ms/sample - loss: 0.4215 - accuracy: 0.5542 - val_loss: 0.4034 - val_accuracy: 0.5657
Epoch 2/30
5376/5471 [============================>.] - ETA: 9s - loss: 0.3972 - accuracy: 0.5670 
Epoch 00002: val_loss improved from 0.40345 to 0.39490, saving model to /kaggle/working/best_model.hdf5
5471/5471 [==============================] - 561s 103ms/sample - loss: 0.3974 - accuracy: 0.5665 - val_loss: 0.3949 - val_accuracy: 0.5700
Epoch 3/30
5376/5471 [============================>.] - ETA: 9s - loss: 0.3874 - accuracy: 0.5715 
Epoch 00003: val_loss improved from 0.39490 to 0.38726, saving model to /kaggle/working/best_model.hdf5
5471/5471 [==============================] - 562s 103ms/sample - loss:

In [61]:
# del model
# gc.collect()
#dfhfdfhf00hthgfhgffh111dxgjfjfhhkghchrthfghfdfghjjhudtkdfgdgnfg

In [62]:
preds = model.predict([test_question_title, test_question_body, test_answer, X_test[X_test.columns[3:]]],
                      batch_size=128, verbose=1
                     )

476/476 [==============================] - 21s 43ms/sample


In [63]:
preds, preds.shape

(array([[0.9177881 , 0.712342  , 0.08738184, ..., 0.05983028, 0.8226221 ,
         0.9065708 ],
        [0.8969238 , 0.6176703 , 0.00853604, ..., 0.11990452, 0.24806282,
         0.89180744],
        [0.906845  , 0.6542593 , 0.06537297, ..., 0.05650342, 0.7727715 ,
         0.89626145],
        ...,
        [0.8928832 , 0.5013012 , 0.02730969, ..., 0.14178583, 0.49215448,
         0.865542  ],
        [0.9090668 , 0.65115255, 0.12713924, ..., 0.0453603 , 0.830601  ,
         0.8912023 ],
        [0.9022551 , 0.61997235, 0.0084469 , ..., 0.1086736 , 0.17629915,
         0.8747628 ]], dtype=float32), (476, 30))

In [64]:
subm = test = pd.read_csv('../input/google-quest-challenge/sample_submission.csv', header=0,encoding='utf-8')
print(subm.shape)
subm.head()

(476, 31)


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [65]:
# targets=['question_asker_intent_understanding',
#        'question_body_critical', 'question_conversational',
#        'question_expect_short_answer', 'question_fact_seeking',
#        'question_has_commonly_accepted_answer',
#        'question_interestingness_others', 'question_interestingness_self',
#        'question_multi_intent', 'question_not_really_a_question',
#        'question_opinion_seeking', 'question_type_choice',
#        'question_type_compare', 'question_type_consequence',
#        'question_type_definition', 'question_type_entity',
#        'question_type_instructions', 'question_type_procedure',
#        'question_type_reason_explanation', 'question_type_spelling',
#        'question_well_written', 'answer_helpful',
#        'answer_level_of_information', 'answer_plausible', 'answer_relevance',
#        'answer_satisfaction', 'answer_type_instructions',
#        'answer_type_procedure', 'answer_type_reason_explanation',
#        'answer_well_written']

targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'    
    ]

In [66]:
for col_index, col in enumerate(targets):
    subm[col] = preds[:, col_index]

In [67]:
subm.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.917788,0.712342,0.087382,0.615233,0.789907,0.696748,0.645133,0.613797,0.342057,...,0.897086,0.907231,0.662985,0.958493,0.961694,0.829056,0.081691,0.059830,0.822622,0.906571
1,46,0.896924,0.617670,0.008536,0.724237,0.759592,0.895591,0.602417,0.379775,0.101342,...,0.796381,0.935397,0.592278,0.966603,0.958511,0.863689,0.816953,0.119905,0.248063,0.891807
2,70,0.906845,0.654259,0.065373,0.709370,0.828192,0.794656,0.619522,0.509468,0.224787,...,0.849624,0.888496,0.596280,0.951036,0.956810,0.829755,0.119774,0.056503,0.772771,0.896261
3,132,0.876939,0.405517,0.002293,0.709570,0.735568,0.924571,0.565191,0.348821,0.116278,...,0.717910,0.931602,0.615061,0.969252,0.981842,0.908628,0.881141,0.142884,0.324065,0.886404
4,200,0.930147,0.651918,0.106463,0.769008,0.702810,0.676741,0.644609,0.591195,0.222067,...,0.885128,0.867112,0.640261,0.965316,0.945469,0.835020,0.192815,0.077056,0.567542,0.899769


In [68]:
subm.to_csv("submission.csv", index = False)

In [69]:
# del model
# gc.collect()

In [70]:
# os.remove('/kaggle/working/training.png')